In [178]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss

from crepes import ConformalPredictiveSystem
from crepes.extras import binning, DifficultyEstimator

from calibrated_explanations import  VennAbers, __version__
from calibrated_explanations.utils import transform_to_numeric
from time import time

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

print(f"calibrated_explanations {__version__}")



calibrated_explanations v0.3.1


In [180]:

datasets = {'COMPAS': 'classification', 'adult': 'classification', 'german': 'classification', 'boston': 'regression', 'CaC': 'regression'}
results = {}
for name, type in datasets.items():
    results[name] = {}
    results[name]['type'] = type
    for category in ['mondrian', 'mondrian_x', 'model_split']:
        results[name][category] = {'time': 0, 'base': 0, 'rand': 0, 'race': 0, 'sex': 0, 'age': 0, 'edu': 0, 'crm': 0}

num_rep = 10
kf = KFold(n_splits=10)

# COMPAS
Data preprocessing from [this notebook](https://colab.research.google.com/github/pair-code/what-if-tool/blob/master/WIT_COMPAS_with_SHAP.ipynb#scrollTo=KF00pJvkeicT).

In [181]:
df = pd.read_csv('https://storage.googleapis.com/what-if-tool-resources/computefest2019/cox-violent-parsed_filt.csv')
print(df.shape)
# Preprocess the data

# Filter out entries with no indication of recidivism or no compass score
df = df[df['is_recid'] != -1]
df = df[df['decile_score'] != -1]


# Make the COMPASS label column numeric (0 and 1), for use in our model
df['score_text'] = np.where(df['score_text'] == 'Low', 'Low', 'Not Low')

target = 'score_text'

(18316, 40)


In [182]:
features_to_keep = ['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'score_text']
# 'age_cat', 'is_recid', 'vr_charge_desc',  'c_charge_desc',, 'is_violent_recid''vr_charge_degree', 'c_charge_degree', 'decile_score', 
df = df[features_to_keep]
df, categorical_features, categorical_labels, target_labels, mappings = transform_to_numeric(df, target)


In [183]:
print(df.columns)
race = 2
age = 1
sex = 0

Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'score_text'],
      dtype='object')


In [184]:
num_to_test = 20 # number of instances to test

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop([target],axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test

X, y = Xd.values, yd.values

In [185]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()

        model.fit(X_train,y_train)
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(X_train, a, axis=1)
                x_cal = np.delete(X_cal, a, axis=1)
                x_test = np.delete(X_test, a, axis=1)
                model = RandomForestClassifier()
                model.fit(x_train,y_train)
            else:
                x_train = X_train
                x_cal = X_cal
                x_test = X_test
                
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results['COMPAS']['mondrian_x']['time'] = time() - time_mondrian_x
results['COMPAS']['mondrian_x']['struct'] = mondrian_x

In [186]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()

        model.fit(X_train,y_train)
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results['COMPAS']['mondrian']['time'] = time() - time_mondrian
results['COMPAS']['mondrian']['struct'] = mondrian

In [187]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, age]
attribute_names = [base, rand, 'race', 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []
    
time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results['COMPAS']['model_split']['time'] = time() - time_split
results['COMPAS']['model_split']['struct'] = model_split

In [188]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [189]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results['COMPAS']['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results['COMPAS']['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results['COMPAS']['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0073 	 0.0072 	 0.0073
rand width: 	 0.0222 	 0.0220 	 0.0200
race width: 	 0.0186 	 0.0186 	 0.0181
sex width: 	 0.0112 	 0.0111 	 0.0111
age width: 	 0.0279 	 0.0207 	 0.0208


In [190]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.7738 	 0.7736 	 0.7734 	 0.7733 	 0.7732 	 0.7727
rand Accuracy: 	 0.7719 	 0.7720 	 0.7277 	 0.7733 	 0.7732 	 0.7281
race Accuracy: 	 0.7650 	 0.7729 	 0.7732 	 0.7614 	 0.7732 	 0.7720
sex Accuracy: 	 0.7691 	 0.7730 	 0.7730 	 0.7683 	 0.7732 	 0.7729
age Accuracy: 	 0.7460 	 0.7721 	 0.7713 	 0.7093 	 0.7732 	 0.7722

base LogLoss: 	 0.4657 	 0.4652 	 0.4657 	 0.7931 	 0.7915 	 0.7859
rand LogLoss: 	 0.4697 	 0.4691 	 0.5407 	 0.7931 	 0.7915 	 0.9481
race LogLoss: 	 0.4782 	 0.4644 	 0.4660 	 0.7171 	 0.7915 	 0.8141
sex LogLoss: 	 0.4700 	 0.4655 	 0.4663 	 0.7441 	 0.7915 	 0.7911
age LogLoss: 	 0.5051 	 0.4639 	 0.4686 	 0.6140 	 0.7915 	 0.7881


In [191]:
import pickle

with open('../evaluation/results_mondrian.pkl', 'wb') as file:
    pickle.dump(results, file)


# Adult

In [192]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [193]:
df = X
target = 'Income'
y = y.replace('<=50K.', '<=50K')
y = y.replace('>50K.', '>50K')
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)
print(target_labels)

{0: '<=50K', 1: '>50K'}


In [194]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values

In [195]:
print(df.columns)
edu = 3
race = 8
sex = 9

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Income'],
      dtype='object')


In [196]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':race, 'sex':sex, 'edu':edu}
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(X_train, a, axis=1)
                x_cal = np.delete(X_cal, a, axis=1)
                x_test = np.delete(X_test, a, axis=1)
                model = RandomForestClassifier()
                model.fit(x_train,y_train)
            else:
                x_train = X_train
                x_cal = X_cal
                x_test = X_test
            
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results['adult']['mondrian_x']['time'] = time() - time_mondrian_x
results['adult']['mondrian_x']['struct'] = mondrian_x

In [197]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: None, rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results['adult']['mondrian']['time'] = time() - time_mondrian
results['adult']['mondrian']['struct'] = mondrian

In [198]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, race, sex, edu]
attribute_names = [base, rand, 'race', 'sex', 'edu']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        

        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': X_train[:,race], 'sex': X_train[:,sex], 'edu': X_train[:,edu]}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': X_cal[:,race], 'sex': X_cal[:,sex], 'edu': X_cal[:,edu]}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': X_test[:,race], 'sex': X_test[:,sex], 'edu': X_test[:,edu]}
        
        try:
            for attr in attribute_names:
                for bin in np.unique(train_bins[attr]):
                    X_train_bin = X_train[train_bins[attr] == bin]
                    y_train_bin = y_train[train_bins[attr] == bin]
                    X_cal_bin = X_cal[cal_bins[attr] == bin]
                    y_cal_bin = y_cal[cal_bins[attr] == bin]
                    X_test_bin = X_test[test_bins[attr] == bin]
                    y_test_bin = y_test[test_bins[attr] == bin]
                
                    if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                        continue
                    
                    model = RandomForestClassifier()
                    model.fit(X_train_bin,y_train_bin)
                    uncal = model.predict_proba(X_test_bin)
                    va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                    proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                    model_split['uncal'][attr].append(uncal[:,1])
                    model_split['proba'][attr].append(proba[:,1])
                    model_split['low'][attr].append(low)
                    model_split['high'][attr].append(high)
                    model_split['width'][attr].append(high - low)
                    model_split['y'][attr].append(y_test_bin)
        except Exception as e:
            print(e)
            print(f'attr: {attr} bin: {bin}')    

results['adult']['model_split']['time'] = time() - time_split
results['adult']['model_split']['struct'] = model_split

index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is out of bounds for axis 1 with size 1
attr: edu bin: 14
index 1 is

In [199]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [200]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results['adult']['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results['adult']['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results['adult']['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0030 	 0.0030 	 0.0030
rand width: 	 0.0097 	 0.0098 	 0.0098
race width: 	 0.0068 	 0.0068 	 0.0066
sex width: 	 0.0047 	 0.0047 	 0.0047
edu width: 	 0.0161 	 0.0161 	 0.0152


In [201]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.8554 	 0.8547 	 0.8550 	 0.8555 	 0.8550 	 0.8552
rand Accuracy: 	 0.8543 	 0.8543 	 0.8491 	 0.8555 	 0.8550 	 0.8499
race Accuracy: 	 0.8552 	 0.8545 	 0.8535 	 0.8552 	 0.8550 	 0.8536
sex Accuracy: 	 0.8554 	 0.8548 	 0.8541 	 0.8557 	 0.8550 	 0.8543
edu Accuracy: 	 0.8574 	 0.8556 	 0.8508 	 0.8573 	 0.8550 	 0.8504

base LogLoss: 	 0.3175 	 0.3181 	 0.3183 	 0.3646 	 0.3650 	 0.3676
rand LogLoss: 	 0.3196 	 0.3202 	 0.3306 	 0.3646 	 0.3650 	 0.3633
race LogLoss: 	 0.3183 	 0.3185 	 0.3221 	 0.3722 	 0.3650 	 0.3663
sex LogLoss: 	 0.3167 	 0.3176 	 0.3203 	 0.3710 	 0.3650 	 0.3726
edu LogLoss: 	 0.3164 	 0.3196 	 0.3311 	 0.3588 	 0.3650 	 0.3887


In [202]:
import pickle

with open('../evaluation/results_mondrian.pkl', 'wb') as file:
    pickle.dump(results, file)


# German


In [203]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 

In [204]:
feature_names = statlog_german_credit_data.variables.description[:-1]
feature_names[0] = 'Status'
feature_names[7] = 'Installment rate'
feature_names[15] = 'Existing credits'
feature_names[17] = 'Num people liable'
target_labels = {1: 'Good', 0: 'Bad'}

In [205]:
print(feature_names)
age = 12
sex = 8

0                         Status
1                       Duration
2                 Credit history
3                        Purpose
4                  Credit amount
5          Savings account/bonds
6       Present employment since
7               Installment rate
8        Personal status and sex
9     Other debtors / guarantors
10       Present residence since
11                      Property
12                           Age
13       Other installment plans
14                       Housing
15              Existing credits
16                           Job
17             Num people liable
18                     Telephone
19                foreign worker
Name: description, dtype: object


In [206]:
df = X
target = 'Class'
df[target] = y
df = df.dropna()
df, categorical_features, categorical_labels, _, _ = transform_to_numeric(df, target)

In [207]:
num_to_test = 10 # number of instances to test, one from each class

df = df.sample(frac=1, random_state=42).sort_values(by=[target])
Xd, yd = df.drop(target,axis=1), df[target] 
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

# select test instances from each class and split into train, cal and test
X, y = Xd.values, yd.values
y[y == 2] = 0

In [208]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'sex':sex, 'age':age}
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['proba'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
time_mondrian_x = time()
for attr in attribute_names:
    va[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['proba'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()

        model.fit(X_train,y_train)
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian_x['y'].append(y_test)
        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(X_train, a, axis=1)
                x_cal = np.delete(X_cal, a, axis=1)
                x_test = np.delete(X_test, a, axis=1)
                model = RandomForestClassifier()
                model.fit(x_train,y_train)
            else:
                x_train = X_train
                x_cal = X_cal
                x_test = X_test
            
            uncal = model.predict_proba(x_test)
            va[attr] = VennAbers(model.predict_proba(x_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(x_test, output_interval=True, bins=test_bins[attr])
            mondrian_x['uncal'][attr].append(uncal[:,1])
            mondrian_x['proba'][attr].append(proba[:,1])
            mondrian_x['low'][attr].append(low)
            mondrian_x['high'][attr].append(high)
            mondrian_x['width'][attr].append(high-low)

results['german']['mondrian_x']['time'] = time() - time_mondrian_x
results['german']['mondrian_x']['struct'] = mondrian_x

In [209]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
va = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['proba'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
time_mondrian = time()
for attr in attribute_names:
    va[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['proba'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            uncal = model.predict_proba(X_test)
            va[attr] = VennAbers(model.predict_proba(X_cal), y_cal, model, bins=cal_bins[attr])
            proba, low, high = va[attr].predict_proba(X_test, output_interval=True, bins=test_bins[attr])
            mondrian['uncal'][attr].append(uncal[:,1])
            mondrian['proba'][attr].append(proba[:,1])
            mondrian['low'][attr].append(low)
            mondrian['high'][attr].append(high)
            mondrian['width'][attr].append(high-low)

results['german']['mondrian']['time'] = time() - time_mondrian
results['german']['mondrian']['struct'] = mondrian

In [210]:
# Model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, sex, age]
attribute_names = [base, rand, 'sex', 'age']
model_split = {}
model_split['uncal'] = {}
model_split['proba'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    model_split['uncal'][attr] = []
    model_split['proba'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        age_cal_bin, age_cal_boundaries = binning(X_cal[:,age], bins=5)
        age_train_bin = binning(X_train[:,age], age_cal_boundaries)
        age_test_bin = binning(X_test[:,age], age_cal_boundaries)
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'sex': X_train[:,sex], 'age': age_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'sex': X_cal[:,sex], 'age': age_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'sex': X_test[:,sex], 'age': age_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestClassifier()                
                model.fit(X_train_bin,y_train_bin)
                uncal = model.predict_proba(X_test_bin)
                va = VennAbers(model.predict_proba(X_cal_bin), y_cal_bin, model)
                proba, low, high = va.predict_proba(X_test_bin, output_interval=True)
                model_split['uncal'][attr].append(uncal[:,1])
                model_split['proba'][attr].append(proba[:,1])
                model_split['low'][attr].append(low)
                model_split['high'][attr].append(high)
                model_split['width'][attr].append(high - low)
                model_split['y'][attr].append(y_test_bin)

results['german']['model_split']['time'] = time() - time_split
results['german']['model_split']['struct'] = model_split

In [211]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['proba'][attr] = np.concatenate(mondrian_x['proba'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['proba'][attr] = np.concatenate(mondrian['proba'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['proba'][attr] = np.concatenate(model_split['proba'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [212]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results['german']['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results['german']['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results['german']['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.0437 	 0.0442 	 0.0447
rand width: 	 0.1271 	 0.1280 	 0.1173
sex width: 	 0.0996 	 0.0996 	 0.0928
age width: 	 0.1246 	 0.1252 	 0.1203


In [213]:
print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} Accuracy: \t{np.mean((mondrian_x["proba"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["proba"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["proba"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}', end='')
    print(f' \t{np.mean((mondrian_x["uncal"][attr] > 0.5) == (mondrian_x["y"] == 1)): .4f} \t{np.mean((mondrian["uncal"][attr] > 0.5) == (mondrian["y"] == 1)): .4f} \t{np.mean((model_split["uncal"][attr] > 0.5) == (model_split["y"][attr] == 1)): .4f}')
print()
for attr in attribute_names:
    print(f'{attr} LogLoss: \t{log_loss(mondrian_x["y"], mondrian_x["proba"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["proba"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["proba"][attr]): .4f}', end='')
    print(f' \t{log_loss(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{log_loss(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{log_loss(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
    

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base Accuracy: 	 0.7530 	 0.7503 	 0.7576 	 0.7589 	 0.7543 	 0.7582
rand Accuracy: 	 0.7448 	 0.7442 	 0.7172 	 0.7589 	 0.7543 	 0.7294
sex Accuracy: 	 0.7492 	 0.7450 	 0.7294 	 0.7629 	 0.7543 	 0.7347
age Accuracy: 	 0.7467 	 0.7429 	 0.7274 	 0.7565 	 0.7543 	 0.7338

base LogLoss: 	 0.5120 	 0.5159 	 0.5127 	 0.5043 	 0.5081 	 0.5064
rand LogLoss: 	 0.5261 	 0.5280 	 0.5611 	 0.5043 	 0.5081 	 0.5503
sex LogLoss: 	 0.5201 	 0.5217 	 0.5396 	 0.5086 	 0.5081 	 0.5303
age LogLoss: 	 0.5236 	 0.5256 	 0.5486 	 0.5146 	 0.5081 	 0.5319


In [214]:
import pickle

with open('../evaluation/results_mondrian.pkl', 'wb') as file:
    pickle.dump(results, file)


# Boston

In [215]:
df = pd.read_csv('../data/reg/HousingData.csv', na_values='NA')
# df.head()

In [216]:
feature_names = df.columns[:-1]
print(feature_names)
crm = 0

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')


In [217]:
target = 'MEDV'
df = df.dropna()
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [218]:
num_to_test = 10 # number of instances to test, one from each class

Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

### Standard Regression
Default confidence of 90%

In [219]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'crm':crm}
attribute_names = [base, rand, 'crm']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(X_train, a, axis=1)
                x_cal = np.delete(X_cal, a, axis=1)
                x_test = np.delete(X_test, a, axis=1)
                model = RandomForestRegressor()
                model.fit(x_train,y_train)
                residuals_cal = model.predict(x_cal) - y_cal
                y_test_hat = model.predict(x_test)
                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results['boston']['mondrian_x']['time'] = time() - time_mondrian
results['boston']['mondrian_x']['struct'] = mondrian_x

In [220]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'crm': crm_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results['boston']['mondrian']['time'] = time() - time_mondrian
results['boston']['mondrian']['struct'] = mondrian

In [221]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, crm]
attribute_names = [base, rand, 'crm']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        crm_cal_bin, crm_cal_boundaries = binning(X_cal[:,crm], bins=3)
        crm_train_bin = binning(X_train[:,crm], crm_cal_boundaries)
        crm_test_bin = binning(X_test[:,crm], crm_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'crm': crm_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'crm': crm_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'crm': crm_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()
                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results['boston']['model_split']['time'] = time() - time_split
results['boston']['model_split']['struct'] = model_split

In [222]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [223]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results['boston']['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results['boston']['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results['boston']['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f} ')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 10.5660 	 11.0025 	 10.2621 
rand width: 	 15.5957 	 15.8963 	 18.9047 
crm width: 	 17.9458 	 18.5122 	 17.5247 


In [224]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} MAE: \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')
    print(f' \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["median"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')    
    print(f' \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base MAE: 	 2.3867 	 2.3959 	 2.3894 	 2.3435 	 2.3462 	 2.3477
rand MAE: 	 2.4382 	 2.4308 	 3.3884 	 2.3435 	 2.3462 	 3.2275
crm MAE: 	 2.3993 	 2.4248 	 2.5377 	 2.3469 	 2.3462 	 2.4523
base R2: 	 0.8387 	 0.8413 	 0.8409 	 0.8408 	 0.8439 	 0.8433
rand R2: 	 0.8346 	 0.8378 	 0.7089 	 0.8408 	 0.8439 	 0.7241
crm R2: 	 0.8379 	 0.8381 	 0.8335 	 0.8421 	 0.8439 	 0.8392


In [225]:
import pickle

with open('../evaluation/results_mondrian.pkl', 'wb') as file:
    pickle.dump(results, file)


## Communities and Crimes

In [226]:
# Load the CSV file
df = pd.read_csv('../data/reg/communities.csv')

# Remove columns with missing values
df = df.dropna(axis=1)

# Remove string columns
df = df.select_dtypes(exclude=['object'])

# df.head()


In [227]:
feature_names = df.columns[:-1]
for i, feature_name in enumerate(feature_names):
    print(f'{i}:{feature_name}', end=", ") 
blk = 4

0:state, 1:fold, 2:population, 3:householdsize, 4:racepctblack, 5:racePctWhite, 6:racePctAsian, 7:racePctHisp, 8:agePct12t21, 9:agePct12t29, 10:agePct16t24, 11:agePct65up, 12:numbUrban, 13:pctUrban, 14:medIncome, 15:pctWWage, 16:pctWFarmSelf, 17:pctWInvInc, 18:pctWSocSec, 19:pctWPubAsst, 20:pctWRetire, 21:medFamInc, 22:perCapInc, 23:whitePerCap, 24:blackPerCap, 25:indianPerCap, 26:AsianPerCap, 27:HispPerCap, 28:NumUnderPov, 29:PctPopUnderPov, 30:PctLess9thGrade, 31:PctNotHSGrad, 32:PctBSorMore, 33:PctUnemployed, 34:PctEmploy, 35:PctEmplManu, 36:PctEmplProfServ, 37:PctOccupManu, 38:PctOccupMgmtProf, 39:MalePctDivorce, 40:MalePctNevMarr, 41:FemalePctDiv, 42:TotalPctDiv, 43:PersPerFam, 44:PctFam2Par, 45:PctKids2Par, 46:PctYoungKids2Par, 47:PctTeen2Par, 48:PctWorkMomYoungKids, 49:PctWorkMom, 50:NumIlleg, 51:PctIlleg, 52:NumImmig, 53:PctImmigRecent, 54:PctImmigRec5, 55:PctImmigRec8, 56:PctImmigRec10, 57:PctRecentImmig, 58:PctRecImmig5, 59:PctRecImmig8, 60:PctRecImmig10, 61:PctSpeakEnglOnly,

In [228]:
target = 'ViolentCrimesPerPop'
df, categorical_features, categorical_labels, target_labels, _ = transform_to_numeric(df, target)

In [229]:
Xd, yd = df.drop(target,axis=1), df[target]
X, y = df.drop(target,axis=1).values, df[target].values
no_of_classes = len(np.unique(yd))
no_of_features = Xd.shape[1]
no_of_instances = Xd.shape[0]

In [230]:
# Mondrian with feature exclusion

base = 'base'
rand = 'rand'
attributes = {base:base, rand:rand, 'race':blk}
attribute_names = [base, rand, 'race']
cps = {}
mondrian_x = {}
mondrian_x['uncal'] = {}
mondrian_x['median'] = {}
mondrian_x['low'] = {}
mondrian_x['high'] = {}
mondrian_x['width'] = {}
mondrian_x['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian_x['uncal'][attr] = []
    mondrian_x['median'][attr] = []
    mondrian_x['low'][attr] = []
    mondrian_x['high'][attr] = []
    mondrian_x['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian_x['y'].append(y_test)        
        
        for attr, a in attributes.items():
            if attr not in [base, rand]:
                x_train = np.delete(X_train, a, axis=1)
                x_cal = np.delete(X_cal, a, axis=1)
                x_test = np.delete(X_test, a, axis=1)
                model = RandomForestRegressor()
                model.fit(x_train,y_train)
                residuals_cal = model.predict(x_cal) - y_cal
                y_test_hat = model.predict(x_test)
                
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian_x['uncal'][attr].append(y_test_hat)
            mondrian_x['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian_x['low'][attr].append(values[:,0])
            mondrian_x['high'][attr].append(values[:,2])
            mondrian_x['width'][attr].append(values[:,2]-values[:,0])

results['CaC']['mondrian_x']['time'] = time() - time_mondrian
results['CaC']['mondrian_x']['struct'] = mondrian_x

In [231]:
# Mondrian with all features

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
mondrian = {}
mondrian['uncal'] = {}
mondrian['median'] = {}
mondrian['low'] = {}
mondrian['high'] = {}
mondrian['width'] = {}
mondrian['y'] = []
for attr in attribute_names:
    cps[attr] = None
    mondrian['uncal'][attr] = []
    mondrian['median'][attr] = []
    mondrian['low'][attr] = []
    mondrian['high'][attr] = []
    mondrian['width'][attr] = []

time_mondrian = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        model = RandomForestRegressor()
        model.fit(X_train,y_train)

        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)    
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        cal_bins = {base: None, rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: None, rand: rand_test_bin, 'race': blk_test_bin}
        residuals_cal = model.predict(X_cal) - y_cal
        y_test_hat = model.predict(X_test)
        mondrian['y'].append(y_test)
        
        for attr in attribute_names:
            cps[attr] = ConformalPredictiveSystem()
            cps[attr].fit(residuals_cal, bins=cal_bins[attr])
            values = cps[attr].predict(y_test_hat, bins=test_bins[attr], lower_percentiles=[5, 50], higher_percentiles=[95, 50])
            mondrian['uncal'][attr].append(y_test_hat)
            mondrian['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
            mondrian['low'][attr].append(values[:,0])
            mondrian['high'][attr].append(values[:,2])
            mondrian['width'][attr].append(values[:,2]-values[:,0])

results['CaC']['mondrian']['time'] = time() - time_mondrian
results['CaC']['mondrian']['struct'] = mondrian

In [232]:
# model split based on Mondrian categories

base = 'base'
rand = 'rand'
attributes = [base, rand, blk]
attribute_names = [base, rand, 'race']
cps = {}
model_split = {}
model_split['uncal'] = {}
model_split['median'] = {}
model_split['low'] = {}
model_split['high'] = {}
model_split['width'] = {}
model_split['y'] = {}
for attr in attribute_names:
    cps[attr] = None
    model_split['uncal'][attr] = []
    model_split['median'][attr] = []
    model_split['low'][attr] = []
    model_split['high'][attr] = []
    model_split['width'][attr] = []
    model_split['y'][attr] = []

time_split = time()
for l in range(num_rep):
    np.random.seed = l
    indeces = np.random.permutation(no_of_instances)
    X = X[indeces,:]
    y = y[indeces]
    for train_index, test_index in kf.split(X):
        X_train_cal, X_test = X[train_index], X[test_index]
        y_train_cal, y_test = y[train_index], y[test_index]
        
        X_train, X_cal, y_train, y_cal = train_test_split(X_train_cal, y_train_cal, test_size=0.3, random_state=42)
        
        
        blk_cal_bin, blk_cal_boundaries = binning(X_cal[:,blk], bins=5)
        blk_train_bin = binning(X_train[:,blk], blk_cal_boundaries)  
        blk_test_bin = binning(X_test[:,blk], blk_cal_boundaries)  
        rand_cal_bin, rand_cal_boundaries = binning(np.random.rand(len(y_cal)), bins=5)
        rand_train_bin = binning(np.random.rand(len(y_train)), rand_cal_boundaries)
        rand_test_bin = binning(np.random.rand(len(y_test)), rand_cal_boundaries)
        train_bins = {base: np.ones(len(y_train)), rand: rand_train_bin, 'race': blk_train_bin}
        cal_bins = {base: np.ones(len(y_cal)), rand: rand_cal_bin, 'race': blk_cal_bin}
        test_bins = {base: np.ones(len(y_test)), rand: rand_test_bin, 'race': blk_test_bin}
        
        
        for attr in attribute_names:
            for bin in np.unique(train_bins[attr]):
                X_train_bin = X_train[train_bins[attr] == bin,:]
                y_train_bin = y_train[train_bins[attr] == bin]
                X_cal_bin = X_cal[cal_bins[attr] == bin,:]
                y_cal_bin = y_cal[cal_bins[attr] == bin]
                X_test_bin = X_test[test_bins[attr] == bin,:]
                y_test_bin = y_test[test_bins[attr] == bin]
                
                if len(y_train_bin) == 0 or len(y_cal_bin) == 0 or len(y_test_bin) == 0:
                    continue
                
                model = RandomForestRegressor()

                model.fit(X_train_bin, y_train_bin)
                
                residuals_cal = model.predict(X_cal_bin) - y_cal_bin
                y_test_bin_hat = model.predict(X_test_bin)
            
                cps[attr] = ConformalPredictiveSystem()
                cps[attr].fit(residuals_cal)
                values = cps[attr].predict(y_test_bin_hat, lower_percentiles=[5, 50], higher_percentiles=[95, 50])
                model_split['uncal'][attr].append(y_test_bin_hat)
                model_split['median'][attr].append(np.mean(values[:,[1, 3]], axis=1))
                model_split['low'][attr].append(values[:,0])
                model_split['high'][attr].append(values[:,2])
                model_split['width'][attr].append(values[:,2]-values[:,0])
                model_split['y'][attr].append(y_test_bin)

results['CaC']['model_split']['time'] = time() - time_split
results['CaC']['model_split']['struct'] = model_split

In [233]:
for attr in attribute_names:
    mondrian_x['uncal'][attr] = np.concatenate(mondrian_x['uncal'][attr])
    mondrian_x['median'][attr] = np.concatenate(mondrian_x['median'][attr]) 
    mondrian_x['low'][attr] = np.concatenate(mondrian_x['low'][attr]) 
    mondrian_x['high'][attr] = np.concatenate(mondrian_x['high'][attr]) 
    mondrian_x['width'][attr] = np.concatenate(mondrian_x['width'][attr]) 
    mondrian['uncal'][attr] = np.concatenate(mondrian['uncal'][attr])
    mondrian['median'][attr] = np.concatenate(mondrian['median'][attr]) 
    mondrian['low'][attr] = np.concatenate(mondrian['low'][attr]) 
    mondrian['high'][attr] = np.concatenate(mondrian['high'][attr]) 
    mondrian['width'][attr] = np.concatenate(mondrian['width'][attr]) 
    model_split['uncal'][attr] = np.concatenate(model_split['uncal'][attr])
    model_split['median'][attr] = np.concatenate(model_split['median'][attr]) 
    model_split['low'][attr] = np.concatenate(model_split['low'][attr]) 
    model_split['high'][attr] = np.concatenate(model_split['high'][attr]) 
    model_split['width'][attr] = np.concatenate(model_split['width'][attr]) 
    model_split['y'][attr] = np.concatenate(model_split['y'][attr]) 

mondrian_x['y'] = np.concatenate(mondrian_x['y']) 
mondrian['y'] = np.concatenate(mondrian['y']) 

In [234]:
print('Category\t Mondrian_x\t Mondrian\t Model_split')
for attr in attribute_names:
    results['CaC']['mondrian'][attr] = np.mean(mondrian['width'][attr])
    results['CaC']['mondrian_x'][attr] = np.mean(mondrian_x['width'][attr])
    results['CaC']['model_split'][attr] = np.mean(model_split['width'][attr])
    print(f'{attr} width: \t{np.mean(mondrian_x["width"][attr]): .4f} \t{np.mean(mondrian["width"][attr]): .4f} \t{np.mean(model_split["width"][attr]): .4f} ')

Category	 Mondrian_x	 Mondrian	 Model_split
base width: 	 0.4596 	 0.4608 	 0.4692 
rand width: 	 0.4904 	 0.4935 	 0.5319 
race width: 	 0.4301 	 0.4350 	 0.4491 


In [235]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

print('Category\t Mondrian_x\t Mondrian\t Model_split\t UMondrian_x\t UMondrian\t UModel_split')
for attr in attribute_names:
    print(f'{attr} MAE: \t{mean_absolute_error(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["median"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')
    print(f' \t{mean_absolute_error(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{mean_absolute_error(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{mean_absolute_error(model_split["y"][attr], model_split["uncal"][attr]): .4f}')
for attr in attribute_names:
    print(f'{attr} R2: \t{r2_score(mondrian_x["y"], mondrian_x["median"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["median"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["median"][attr]): .4f}', end='')    
    print(f' \t{r2_score(mondrian_x["y"], mondrian_x["uncal"][attr]): .4f} \t{r2_score(mondrian["y"], mondrian["uncal"][attr]): .4f} \t{r2_score(model_split["y"][attr], model_split["uncal"][attr]): .4f}')

Category	 Mondrian_x	 Mondrian	 Model_split	 UMondrian_x	 UMondrian	 UModel_split
base MAE: 	 0.1025 	 0.1018 	 0.1027 	 0.0950 	 0.0947 	 0.0952
rand MAE: 	 0.1027 	 0.1019 	 0.1092 	 0.0950 	 0.0947 	 0.1011
race MAE: 	 0.1034 	 0.1022 	 0.1048 	 0.0952 	 0.0947 	 0.0969
base R2: 	 0.6346 	 0.6376 	 0.6333 	 0.6483 	 0.6496 	 0.6469
rand R2: 	 0.6332 	 0.6369 	 0.5887 	 0.6483 	 0.6496 	 0.6052
race R2: 	 0.6298 	 0.6332 	 0.6177 	 0.6479 	 0.6496 	 0.6382


## Summary
### Time

In [236]:
categories = ['mondrian_x','mondrian','model_split',]
print('Dataset\t Mondrian_x\t Mondrian\t Model Split\tType')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for category in categories:
        print(f'{results[name][category]["time"]: 0.2f}\t\t', end='')
    print(f'{type}')

Dataset	 Mondrian_x	 Mondrian	 Model Split	Type
COMPAS	 351.88		 156.05		 8892.53		classification
adult	 25967.90		 748.27		 2969.38		classification
german	 65.61		 29.36		 273.06		classification
boston	 101.25		 48.64		 204.55		regression
CaC	 2885.50		 1294.56		 10366.43		regression


### Width
#### Mondrian vs model split

In [237]:
print('', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in categories:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in categories:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in categories:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	 mondrian_x	 mondrian	 model_split	
Dataset	 base		 base		 base		 rand		 rand		 rand		 race		 race		 race		 sex		 sex		 sex		 age		 age		 age		 edu		 edu		 edu		 crm		 crm		 crm		Type
COMPAS	 0.007		 0.007		 0.007		 0.022		 0.022		 0.020		 0.019		 0.019		 0.018		 0.011		 0.011		 0.011		 0.028		 0.021		 0.021		 -		 -		 -		 -		 -		 -		classification
adult	 0.003		 0.003		 0.003		 0.010		 0.010		 0.010		 0.007		 0.007		 0.007		 0.005		 0.005		 0.005		 -		 -		 -		 0.016		 0.016		 0.015		 -		 -		 -		classification
german	 0.044		 0.044		 0.045		 0.127		 0.128		 0.117		 -		 -		 -		 0.100		 0.100		 0.093		 0.125		 0.125		 0.120		 -		 -		 -		 -		 -		 -		classification
boston	 10.566		 11.003		 10.262		 15.596		 15.896		 18.905		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 -		 17.946		 1

#### Only Mondrian

In [238]:
print('', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {category}', end='\t')
print('\nDataset', end='\t')
for attr in results[name]['mondrian'].keys():
    if attr in ['time', 'struct']:
        continue
    for category in ['mondrian']:
        print(f' {attr}', end='\t\t')
print('Type')
for name, type in datasets.items():
    print(f'{name}\t', end='')
    for attr in results[name]['mondrian'].keys():
        if attr in ['time', 'struct']:
            continue
        for category in ['mondrian']:
            val = results[name][category][attr]
            print(f'{val: 0.3f}', end='\t\t') if val > 0 else print(' -', end='\t\t')
    print(f'{type}')

	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	 mondrian	
Dataset	 base		 rand		 race		 sex		 age		 edu		 crm		Type
COMPAS	 0.007		 0.022		 0.019		 0.011		 0.021		 -		 -		classification
adult	 0.003		 0.010		 0.007		 0.005		 -		 0.016		 -		classification
german	 0.044		 0.128		 -		 0.100		 0.125		 -		 -		classification
boston	 11.003		 15.896		 -		 -		 -		 -		 18.512		regression
CaC	 0.461		 0.493		 0.435		 -		 -		 -		 -		regression


In [239]:
import pickle

with open('../evaluation/results_mondrian.pkl', 'wb') as file:
    pickle.dump(results, file)
